In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

2025-07-09 14:01:36.727191: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 14:01:36.740096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752051696.752948    2745 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752051696.757070    2745 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752051696.768440    2745 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Data

In [2]:
X = np.loadtxt(open("data/small_movies_X.csv", "rb") , delimiter = ",")
W = np.loadtxt(open("data/small_movies_W.csv", "rb") , delimiter = ",")
b = np.loadtxt(open("data/small_movies_b.csv", "rb") , delimiter = ",")
Y = np.loadtxt(open("data/small_movies_Y.csv", "rb") , delimiter = ",")
R = np.loadtxt(open("data/small_movies_R.csv", "rb") , delimiter = ",")

In [3]:
print("X: ", X.shape)
print("W: ", W.shape)
print("b: ", b.shape)
print("Y: ", Y.shape)
print("R: ", R.shape)

X:  (4778, 10)
W:  (443, 10)
b:  (443,)
Y:  (4778, 443)
R:  (4778, 443)


In [4]:
b = b.reshape(1, -1)
print("b =", b.shape)

b = (1, 443)


In [5]:
num_movies, num_features = X.shape
num_users = W.shape[0]

print("Movies: " , num_movies)
print("Users: " , num_users)
print("Features: " , num_features)

Movies:  4778
Users:  443
Features:  10


In [6]:
movie = Y[0,:]

In [7]:
rated = R[0,:] == 1

In [8]:
np.mean(movie[rated])

np.float64(3.4)

### FIltering

In [9]:
def cofiCostFunc(X,W,b,Y,R, lambda_=0):
    cost = (((tf.linalg.matmul(X, tf.transpose(W)) + b) - Y) * R)**2 / 2
    reg_w = (lambda_/2)*tf.reduce_sum(W**2)
    reg_x = (lambda_/2)*tf.reduce_sum(X**2)
    return tf.reduce_sum(cost) + reg_w + reg_x

In [10]:
users1, movies1, features1 = 4, 5, 3
X_r = X[:movies1, :features1]
W_r = W[:users1, :features1]
b_r = b[0, :users1].reshape(1, -1)
Y_r = Y[:movies1, :users1]
R_r = R[:movies1, :users1]

In [11]:
b_r.shape

(1, 4)

In [12]:
cofiCostFunc(X_r, W_r, b_r, Y_r, R_r)

I0000 00:00:1752051700.173774    2745 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


<tf.Tensor: shape=(), dtype=float64, numpy=13.670725805579917>

In [13]:
cofiCostFunc(X_r, W_r, b_r, Y_r, R_r, 1.5)

<tf.Tensor: shape=(), dtype=float64, numpy=28.09383799145902>

### Preprocessing

In [14]:
my_ratings = np.zeros(num_movies)

my_ratings[2700] = 5 
my_ratings[2609] = 2
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)

In [15]:
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

In [16]:
df = pd.read_csv("data/small_movie_list.csv", header=0, index_col=0)
df.head()

,mean rating,number of ratings,title
0,3.400000,5,"Yards, The (2000)"
1,3.250000,6,Next Friday (2000)
2,2.000000,4,Supernova (2000)
3,2.000000,4,Down to You (2000)
4,2.672414,29,Scream 3 (2000)


In [17]:
movie_list = df["title"].tolist()

In [18]:
movie_list[:5]

['Yards, The (2000)',
 'Next Friday (2000)',
 'Supernova (2000)',
 'Down to You (2000)',
 'Scream 3 (2000)']

In [19]:
for i in my_rated:
    print(f"Rated {my_ratings[i]} for {movie_list[i]}")

Rated 5.0 for Shrek (2001)
Rated 5.0 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for Incredibles, The (2004)
Rated 2.0 for Persuasion (2007)
Rated 5.0 for Toy Story 3 (2010)
Rated 3.0 for Inception (2010)
Rated 1.0 for Louis Theroux: Law & Disorder (2008)
Rated 1.0 for Nothing to Declare (Rien à déclarer) (2010)


In [20]:
Y = np.c_[my_ratings, Y]
R = np.c_[(my_ratings!=0).astype(int), R]

In [21]:
average_ratings = np.sum(Y, axis=1) / (np.sum(R, axis=1) + 1e-12)
average_ratings

array([3.4 , 3.25, 2.  , ..., 3.5 , 3.5 , 3.5 ], shape=(4778,))

In [22]:
average_ratings = average_ratings.reshape(-1,1)

In [23]:
Ynorm = Y - np.multiply(average_ratings,R)
Ynorm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4778, 444))

### Training

In [24]:
tf.random.set_seed(1234)

optimizer = tf.keras.optimizers.Adam(0.1)

W = tf.Variable(tf.random.normal((Y.shape[1], 100), dtype=tf.float64), name='W')
X = tf.Variable(tf.random.normal((Y.shape[0], 100), dtype=tf.float64), name='X')
b = tf.Variable(tf.random.normal((1, Y.shape[1]), dtype=tf.float64), name='b')

In [25]:
for i in range(200):
    with tf.GradientTape() as tape:
        cost = cofiCostFunc(X, W, b, Ynorm, R, 1)
        grads = tape.gradient(cost, [X, W, b])
        optimizer.apply_gradients(zip(grads,[X, W, b]))
    if i % 20 == 0:
        print(f"Iteration: {i}, Cost: {cost:0.2f}")

Iteration: 0, Cost: 2321191.27
Iteration: 20, Cost: 136169.26
Iteration: 40, Cost: 51863.65
Iteration: 60, Cost: 24599.03
Iteration: 80, Cost: 13630.56
Iteration: 100, Cost: 8487.74
Iteration: 120, Cost: 5807.76
Iteration: 140, Cost: 4311.64
Iteration: 160, Cost: 3435.26
Iteration: 180, Cost: 2902.09


### Recomendations

In [26]:
pred = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

In [27]:
pred = pred + average_ratings

In [35]:
my_pred = pred[:, 0]

In [37]:
ix = tf.argsort(my_pred, direction= "DESCENDING")

In [38]:
ix

<tf.Tensor: shape=(4778,), dtype=int32, numpy=
array([1150,  246,  929, ..., 3680, 2644, 1209],
      shape=(4778,), dtype=int32)>

In [40]:
for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_pred[j]:0.2f} for movie {movie_list[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_pred[i]:0.2f} for {movie_list[i]}')

Predicting rating 4.49 for movie My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicting rating 4.48 for movie Martin Lawrence Live: Runteldat (2002)
Predicting rating 4.48 for movie Memento (2000)
Predicting rating 4.47 for movie Delirium (2014)
Predicting rating 4.47 for movie Laggies (2014)
Predicting rating 4.47 for movie One I Love, The (2014)
Predicting rating 4.46 for movie Particle Fever (2013)
Predicting rating 4.45 for movie Eichmann (2007)
Predicting rating 4.45 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.45 for movie Into the Abyss (2011)


Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.13 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predic